In [ ]:
from sapphire import *

In [ ]:
mgmt_ip = '10.127.190.136'
password = 'nbv_12345'

mgmt_ip = '10.127.125.228'
password = 'nbv_12345'

mgmt_ip = '10.127.119.98'
password = 'nbv_3124'

mgmt_ip = '172.25.169.24'
password = 'nbv_12345'

mgmt_ip = '10.197.155.82'
password = 'n'




mgmt_ip = '10.127.190.134'

mgmt_ip = '10.127.190.138'

mgmt_ip = '10.127.125.219'


password = 'nbv_12345'

switch = Switch(mgmt_ip, password)

#vsan = int(input('Please enter the VSAN of interest : '))
vsan = 1000

In [ ]:
switch

In [ ]:
fabric = Fabric(switch)

In [ ]:
fabric.search_devices(vsan = vsan)

In [ ]:
for device in fabric.devices:
    device.print_details()
    print(device.show_flogi_database(vsan = vsan))

In [ ]:
switch.show_int_brief_fc()

In [ ]:
switch.show_flogi_database(vsan = vsan)

In [ ]:
switch.show_flogi_database(vsan = vsan)

In [ ]:
for key,value in switch.fc_interface.items():
    print(key, value)

In [ ]:
switch.fc_interface['fc1/1'].show()

In [ ]:
switch.fc_interface['fc1/17'].__dict__

In [ ]:
#switch.fc_interface['fc1/17'].flap()

In [ ]:
switch.fc_interface['fc1/17'].show_brief()

In [ ]:
switch.show_zoneset_active(vsan = vsan)

In [ ]:
zone_dict = switch.get_zoneset_active(vsan = vsan)
print(zone_dict)

In [ ]:
df = pd.DataFrame.from_dict(zone_dict, orient = 'index')
print(df.shape)
df.head(df.shape[0])

In [ ]:
switch.show_fcns_database(vsan = vsan)

In [ ]:
switch.show_flogi_database(vsan = vsan)

In [ ]:
# See https://neo4j.com/developer/aura-connect-driver/ for Aura specific connection URL.
scheme = "neo4j"  # Connecting to Aura, use the "neo4j+s" URI scheme
host_name = "localhost"
port = 7687
url = "{scheme}://{host_name}:{port}".format(scheme=scheme, host_name=host_name, port=port)
user = "admin"
password = "admin"
app = App(url, user, password)

In [ ]:
for switch in fabric.devices:
    switch.print_details()
    if 'MDS' in switch.descr:
        continue
    switch.show_int_brief_fc()
    for interface in switch.fc_interface:
        print ('Creating ports from {0}, interface : {1}'.format(switch.switchname, interface))
        app.create_port_node(switch.switchname, interface)
        
    switch.show_int_brief_vfc()
    for interface in switch.vfc_interface:
        print ('Creating ports from {0}, interface : {1}'.format(switch.switchname, interface))
        app.create_port_node(switch.switchname, interface)
        
    switch.show_int_brief_sanpo()
    for interface in switch.sanpo_interface:
        print ('Creating ports from {0}, interface : {1}'.format(switch.switchname, interface))
        app.create_port_node(switch.switchname, interface)

In [ ]:
# Not supporting E links currently
'''
for switch in fabric.devices:
    switchname = switch.switchname
    print(switch.switchname, switch.mgmt_ip)
    peers = switch.get_topology(vsan = 1000)
    print(peers.head())
    for i in range(peers.shape[0]):
        peer_switch = peers['Switch Name'][i]
        app.create_link(switchname, peer_switch, 'E')
'''


In [ ]:
for switch in fabric.devices:
    switchname = switch.switchname
    flogi_df = switch.get_flogi_database(vsan = vsan)
    print(flogi_df.head())
    for i in range(flogi_df.shape[0]):
        fcid = flogi_df['FCID'][i]
        port = flogi_df['Interface'][i]
        switch_port = switchname + '_' + port
        app.create_link(switch_port, fcid, 'N')

In [ ]:
from tqdm import tqdm 

# Getting zone config
zone_dict = switch.get_zoneset_active(vsan = vsan)
zone_df = pd.DataFrame.from_dict(zone_dict, orient = 'index')


for index, row in tqdm(zone_df.iterrows(), total = zone_df.shape[0]): 
    #print('Iterating over devices under zone {}'.format(index))
    concerned_ports = []
    for i in range(len(row)):
        if row[i] == None:
            #Ignoring empty cells
            continue
            
        # row[i][0] is the member type, and row[i][1] is the value.    
        switch_port = get_switch_port(fabric, vsan, row[i][0], row[i][1])
        if switch_port != None:
            concerned_ports.append(switch_port)
            
    #Now, create links between all the devices under the same zone.
    for i in range(len(concerned_ports)):
        for j in range(i+1, len(concerned_ports)):
            #print('Adding virtual link between {} and {}'.format(concerned_ports[i], concerned_ports[j]))
            app.create_link(concerned_ports[i], concerned_ports[j], 'V')

            

In [ ]:
app.close()